## Task Description

Navigate to the list of faculty in the Anesthesiology Department (be aware that each website is very different. For instance, the links for Upstate and Westchester are already linked to the Anesthesiology program, but you will have to navigate to the faculty page; the New Mexico website has to be filtered first in order to see just the anesthesiology faculty). 


Once you find the appropriate pages with the faculty listed, I would like you to create an excel file, where each institution as its own sheet. There should be four headers for each sheet: First Name, Last Name, Email, Error. The first name and last name of a clinician should be in the appropriate cell as well as their email (if scrapable). Error should remain blank for now. There should not be any other text in the first name or last name columns (no MD, DO, or any titles, just a single name in each). 

In [8]:
# Install dependencies

!pip install beautifulsoup4
!pip install pandas
!pip install xlsxwriter

In [52]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Define URLs for each institution
urls = {
    #"UNM": "https://hsc.unm.edu/directory/",
    #"Upstate": "https://www.upstate.edu/anesthesiology/about-us/index.php",
    "Westchester": "https://www.westchestermedicalcenter.org/anesthesiology-residency-program",
}

# Create an empty dictionary to store dataframes for each institution
dfs = {}

# Function to extract faculty information from a given URL
def extract_faculty_data(url, institution_name):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    faculty_data = [] # Initialize an empty list to store data

    if "hsc.unm.edu" in url:
        # Logic for University of New Mexico
        # ... (Add your specific code here to extract faculty data and append to faculty_data list)
        pass

    elif "upstate.edu" in url:
        # Logic for Upstate Medical University
        # ... (Add your specific code here to extract faculty data and append to faculty_data list)
        pass

    elif "westchestermedicalcenter.org" in url:
        # Logic for Westchester Medical Center
        faculty_heading = soup.find("h2", string="Faculty")

        if faculty_heading:
            faculty_container = faculty_heading.parent
            #print(f"Debug (faculty_container): {faculty_container}")

            # Extract faculty information
            faculty_data = []
            strong_tags = faculty_container.find_all("strong")  
            for strong_tag in strong_tags:
                split_text = strong_tag.text.split(",")
                if len(split_text) > 1:
                    full_name = split_text[0]
                    full_name = full_name.split(".")
                    if len(full_name) > 1:
                        first_name = full_name[0] + "."
                        last_name = full_name[1]
                        faculty_data.append([first_name, last_name, "", ""]) 
                    else:
                        full_name = split_text[0].split(" ")
                        first_name = full_name[0]
                        last_name = full_name[1]
                        faculty_data.append([first_name, last_name, "", ""]) 
        else:
            print(f"Error: 'Faculty' heading not found on the {name} webpage.")

    else:
        print(f"Unsupported website: {url}")
        return

    # Create a pandas DataFrame
    df = pd.DataFrame(faculty_data)

    # Set the column headers (order matters)
    df.columns = ["First Name", "Last Name", "Email", "Error"]

    display(df)

    return df

# Iterate through URLs and extract faculty data for each institution
for institution_name, url in urls.items():
    dfs[institution_name] = extract_faculty_data(url, institution_name)

file_name = "anesthesiology_faculty.xlsx"
with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
    # Write each DataFrame to a separate sheet in the Excel file
    for institution_name, df in dfs.items():
        df.to_excel(writer, sheet_name=institution_name, index=False)
        print (f"Saved {institution_name} data to {file_name}")


,First Name,Last Name,Email,Error
0,Peter J.,Panzica,,
1,A.,Elisabeth Abramowicz,,
2,Sarah,Smith,,
3,Irim,Salik,,
4,Richard,Yeom,,
5,Nitin,Sekhri,,
6,Ashley M.,Kelley,,
7,Iyabo,Muse,,
8,Garret M.,Weber,,
9,Michael,Rahimi,,


In [43]:
#debug cell

url = "https://www.westchestermedicalcenter.org/anesthesiology-residency-program"
institution_name = "Westchester"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

faculty_data = []  # Initialize an empty list to store data
faculty_heading = soup.find("h2", string="Faculty")

faculty_container = faculty_heading.parent
#print(f"Faculty_container (debug):\n{faculty_container}")

strong_tags = faculty_container.find_all("strong")  
for strong_tag in strong_tags:
    split_text = strong_tag.text.split(",")
    if len(split_text) > 1:
        full_name = split_text[0]
        full_name = full_name.split(".")
        if len(full_name) > 1:
            first_name = full_name[0] + "."
            last_name = full_name[1]
            faculty_data.append([first_name, last_name, "", ""]) 
        else:
            full_name = split_text[0].split(" ")
            first_name = full_name[0]
            last_name = full_name[1]
            faculty_data.append([first_name, last_name, "", ""]) 

# Create a pandas DataFrame
df = pd.DataFrame(faculty_data)

# Set the column headers (order matters)
df.columns = ["First Name", "Last Name", "Email", "Error"] 

display(df)

,First Name,Last Name,Email,Error
0,Peter J.,Panzica,,
1,A.,Elisabeth Abramowicz,,
2,Sarah,Smith,,
3,Irim,Salik,,
4,Richard,Yeom,,
5,Nitin,Sekhri,,
6,Ashley M.,Kelley,,
7,Iyabo,Muse,,
8,Garret M.,Weber,,
9,Michael,Rahimi,,
